<a href="https://colab.research.google.com/github/bhavrish/CVD-Classifer/blob/main/Models/Neural%20Nets/Neural_Nets_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
import pandas as pd
import matplotlib.pyplot as plt
import math
from sklearn.metrics import classification_report
from sklearn import metrics
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_validate
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [ ]:
grid_params = {
    "batch_size": [20, 25, 50, 100, 200], 
    "nb_epoch": [100, 200, 300, 400, 500],
    "unit": [2, 4, 8, 16, 32, 64]
}

In [2]:
# https://www.geeksforgeeks.org/hyperparameter-tuning-using-gridsearchcv-and-kerasclassifier/

def nn_builder(unit):
    # 2 hidden layers, 1 input layer, 1 output layer
    model = keras.models.Sequential()
    model.add(keras.layers.Dense(units=unit, activation='relu'))
    model.add(keras.layers.Dense(units=unit, activation='relu'))
    model.add(keras.layers.Dense(units=1, activation='sigmoid'))
    model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
    return model

In [ ]:
Clusters = ["CLUSTER0", "CLUSTER1", "CLUSTER2", "CLUSTER3", "CLUSTER4", "CLUSTER5"]
accuracyDict = {}
numEstimatorsDict = {}

for clust in Clusters:
    # Preprocessing
    path_k = "/content/" + clust
    df = pd.read_csv(path_k, delimiter=',')
    df = df.iloc[:,2:] # drop original/new ids
    df_feat = df.iloc[:,:-1] # everything but last column
    y = df.iloc[:,-1] #last column
    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(df_feat)
    X = pd.DataFrame(scaled_features)
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    
    # Create model
    NN = KerasClassifier(build_fn=nn_builder)
    
    # Train model
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
    gs = GridSearchCV(NN, grid_params, verbose= 1, cv= 3, n_jobs= -1)
    gs.fit(X_train, y_train)

    # Store accuracies
    y_pred = gs.predict(X_test)
    accuracyDict[clust] = metrics.accuracy_score(y_test, y_pred)

    # Store best # of estimtators
    numEstimatorsDict[clust] = gs.best_params_
    
print(accuracyDict)
print(numEstimatorsDict)

Fitting 3 folds for each of 150 candidates, totalling 450 fits


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


94/94 [==============================] - 1s 1ms/step - loss: 0.5959 - accuracy: 0.7236


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


Fitting 3 folds for each of 150 candidates, totalling 450 fits
209/209 [==============================] - 1s 2ms/step - loss: 0.6235 - accuracy: 0.6582


...

Fitting 3 folds for each of 150 candidates, totalling 450 fits
143/143 [==============================] - 1s 2ms/step - loss: 0.5673 - accuracy: 0.7261


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


Fitting 3 folds for each of 150 candidates, totalling 450 fits
207/207 [==============================] - 1s 2ms/step - loss: 0.5896 - accuracy: 0.6967


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:18: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


Fitting 3 folds for each of 150 candidates, totalling 450 fits
934/934 [==============================] - 2s 2ms/step - loss: 0.5768 - accuracy: 0.6986
{'CLUSTER0': 0.8131677018633541, 'CLUSTER1': 0.6868347338935574, 'CLUSTER2': 0.7440147329650092, 'CLUSTER3': 0.7636958299264105, 'CLUSTER4': 0.6998872604284103, 'CLUSTER5': 0.704338042255282}
{'CLUSTER0': {'batch_size': 100, 'nb_epoch': 400, 'unit': 16}, 'CLUSTER1': {'batch_size': 20, 'nb_epoch': 400, 'unit': 64}, 'CLUSTER2': {'batch_size': 20, 'nb_epoch': 200, 'unit': 64}, 'CLUSTER3': {'batch_size': 20, 'nb_epoch': 100, 'unit': 64}, 'CLUSTER4': {'batch_size': 50, 'nb_epoch': 400, 'unit': 64}, 'CLUSTER5': {'batch_size': 20, 'nb_epoch': 300, 'unit': 64}}


In [4]:
# Save models

from sklearn.pipeline import make_pipeline
import pickle

nn_best_clusters = [1, 3]
best_epochs = [400, 100]

pos = 0

for k in nn_best_clusters:
    inputPath = "/content/Cluster" + str(k)
    outputPath = "./Cluster_" + str(k) + "_NN_pipeline.pkl"
    
    df = pd.read_csv(inputPath, delimiter=',')
    df = df.iloc[:,2:] # drop original/new ids

    X = df.iloc[:,:-1] # everything but last column
    y = df.iloc[:,-1] #last column

    scaler = StandardScaler()
    scaled_features = scaler.fit_transform(X)
    X = pd.DataFrame(scaled_features)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

    NN = KerasClassifier(build_fn=nn_builder, batch_size=20, nb_epoch=best_epochs[pos], unit=64)
    NN.fit(X_train, y_train)
    
    # Create pipeline
    pipeline = make_pipeline(scaler, NN)
    
    # Write model to file
    pickle.dump(pipeline, open(outputPath, 'wb')) 

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


209/209 [==============================] - 3s 5ms/step - loss: 0.6149 - accuracy: 0.6685
INFO:tensorflow:Assets written to: ram://11bc2ef2-11ce-4863-af93-c467566b86f2/assets


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:27: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead.


143/143 [==============================] - 2s 4ms/step - loss: 0.5684 - accuracy: 0.7236
INFO:tensorflow:Assets written to: ram://3b341273-bcf4-418f-852a-c3789a5aa673/assets
